In [1]:
import sys
sys.path.insert(0, '../')


import pandas as pd
import numpy as np

import dsm.prognostics.datasets as datasets
import dsm.prognostics.utilities as utilities

from dsm import DeepSurvivalMachines, DeepConvolutionalSurvivalMachines

from torch import nn 


train_data, test_data = datasets.load_turbofan(cmapss_folder='../../CMAPSSData/', 
                                               experiment=1, 
                                               windowsize=30,
                                               flatten=False,
                                               sequential=False,
                                               test_last=True)

x_tr, t_tr, e_tr = train_data
x_te, t_te, e_te = test_data

t_tr[t_tr>125] = 125


../dsm/prognostics/datasets.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(x), np.array(t), np.array(e), ft_cols, norm_tuple
shape: (100, 1, 15, 30) (1, 15, 30)
../dsm/prognostics/datasets.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(x), np.array(t), np.array(e), ft_cols, norm_tuple


In [2]:
import torch

#x_tr = torch.from_numpy(x_tr)
# x_tr = x_tr.unsqueeze(-1)
# x_tr = torch.transpose(x_tr, 1,3)

In [3]:
print(x_tr.shape)

(17731, 15, 30)


In [35]:

# class CNN(nn.Module):
    
#     def __init__(self, hidden, output):
#         super(CNN, self).__init__()

#         self.zeropad = nn.ZeroPad2d((0,0,0,9))
#         self.conv1 = nn.Conv2d(1, 10, (10,1), 1,0,1)
#         self.conv2 = nn.Conv2d(10, 10, (10,1),1,0,1)
#         self.conv3 = nn.Conv2d(10, 10, (10,1),1,0,1)
#         self.conv4 = nn.Conv2d(10, 10, (10,1),1,0,1)
#         self.conv5 = nn.Conv2d(10,1,(3,1),1,(1,0),1)
#         self.fc1 = nn.Linear(hidden,output)
#         self.act = nn.Tanh()
        
#     def forward(self,input):
        
#         input = input.unsqueeze(-1)
#         input = torch.transpose(input, 1, 3).contiguous()

#         out = self.zeropad(input)
#         out = self.zeropad(self.act(self.conv1(out)))
#         out = self.zeropad(self.act(self.conv2(out)))
#         out = self.zeropad(self.act(self.conv3(out)))
#         out = self.act(self.conv4(out))
#         out = self.act(self.conv5(out))
#         out = self.fc1(out.view(out.size(0),-1))
        
#         return out

class CNN(nn.Module):
    
    def __init__(self, hidden, output, dropout=0.5):
        super(CNN, self).__init__()

        self.zeropad = nn.ZeroPad2d((0,0,0,9))
        self.conv1 = nn.Conv2d(1, 10, (10,1), 1,0,1)
        self.conv2 = nn.Conv2d(10, 10, (10,1),1,0,1)
        self.conv3 = nn.Conv2d(10, 10, (10,1),1,0,1)
        self.conv4 = nn.Conv2d(10, 10, (10,1),1,0,1)
        self.conv5 = nn.Conv2d(10,1,(3,1),1,(1,0),1)
        self.fc = nn.Linear(hidden, output)
        self.act = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,input):

        input = input.unsqueeze(-1)
        input = torch.transpose(input, 1, 3).contiguous()

        out = self.zeropad(input)
        out = self.dropout(self.zeropad(self.act(self.conv1(out))))
        out = self.dropout(self.zeropad(self.act(self.conv2(out))))
        out = self.dropout(self.zeropad(self.act(self.conv3(out))))
        out = self.dropout(self.act(self.conv4(out)))
        out = self.dropout(self.act(self.conv5(out)))
        out = self.dropout(self.act(self.fc(out.view(out.size(0),-1))))
        
        return out

In [36]:
base_model = CNN(540, 100)

In [37]:
base_model(torch.from_numpy(x_tr).float()).shape

torch.Size([17731, 100])

In [38]:
base_model = CNN(540, 100)
model = DeepConvolutionalSurvivalMachines(hidden=100, distribution='Normal',temp=0.25, embedding=base_model)


In [39]:
model.fit(x_tr, t_tr, e_tr, batch_size=512, learning_rate=1e-3,
          iters=1, val_data=train_data, early_stop=False)

  0%|          | 0/1 [00:00<?, ?it/s]
Parameter containing:
tensor([80.4090, 80.4090, 80.4090], requires_grad=True) Parameter containing:
tensor([41.8006, 41.8006, 41.8006], requires_grad=True)


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 6 1 3 3, but got 3-dimensional input of size [512, 18, 30] instead